# Mutual funds vs. Index drawdowns

Do active managers reduce drawdowns during downturns?

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

indices <- Indices()
mfi <- MutualFundsIndia()

2018 was a bad year. So lets look at the subsequent return profile of the top 5 midcap funds entring 2018 through June-2019.

In [ ]:
lastAumDate <- (mfi$AumSchemewise() %>%
    filter(PERIOD <= as.Date('2018-01-01')) %>%
    summarize(MAX_TS = max(PERIOD)) %>%
    collect())$MAX_TS[1]

lastMetaDate <- (mfi$Meta() %>%
    filter(AS_OF >= lastAumDate) %>%
    summarize(MIN_TS = min(AS_OF)) %>%
    collect())$MIN_TS[1]

print(lastAumDate)
print(lastMetaDate)

topAum <- mfi$Meta() %>% 
    inner_join(mfi$AumSchemewise(), by="SCHEME_CODE") %>%
    filter(PERIOD == lastAumDate & AS_OF == lastMetaDate & CATEGORY %like% '%Mid-Cap%') %>%
    select(SCHEME_CODE, SCHEME_NAME, CATEGORY, AVG_AUM_WO_FOFD) %>%
    arrange(AVG_AUM_WO_FOFD) %>%
    top_n(10)

print(topAum)

funds went through a re-classification exercise later in 2018. we will handpick those that retained their "mid-cap" status after the shuffle.

In [ ]:
schemeCodes <- c(105758, 101539, 100473, 104481, 112932)

startDate <- as.Date("2018-01-01")
endDate <- as.Date("2018-06-30")

indexName <- "NIFTY MIDCAP 150 TR"

#get the daily returns of the funds
navXts <- NULL
for(sc in schemeCodes){
    navs <- data.frame(mfi$NavTimeSeries() %>%
        filter(SCHEME_CODE == scode & TIME_STAMP >= startDate & TIME_STAMP <= endDate) %>%
        select(TIME_STAMP, NAV))
    
    navXts <- merge.xts(navXts, dailyReturn(xts(navs$NAV, navs$TIME_STAMP)))
}

#grab the benchmark
bm <- data.frame(indices$NseTimeSeries() %>%
    filter(NAME == indexName & TIME_STAMP >= startDate & TIME_STAMP <= endDate) %>%
    select(TIME_STAMP, CLOSE))

navXts <- merge.xts(navXts, dailyReturn(xts(bm$NAV, bm$TIME_STAMP)))
names(navXts) <- c(schemeCodes, indexName)

In [ ]:
# plot the cumulative returns
Common.PlotCumReturns(navXts, "Mutual funds vs. Index", "Midcaps vs. 150 TR")


In [ ]:
# legend
topAum %>%
    filter(SCHEME_CODE %in% schemeCodes) %>%
    select(SCHEME_CODE, SCHEME_NAME) %>%
    print()

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)